In [47]:
import os
import sys

In [48]:
sys.path.append('../lib')

In [49]:
import cov_independence_test
import hsic_independence_test
import gen_processes

In [50]:
from importlib import reload

In [51]:
reload(cov_independence_test);
reload(hsic_independence_test);
reload(gen_processes);

In [52]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.ar_model as ar
import torch
torch.set_default_dtype(torch.double)
import sigkernel

In [53]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [54]:
import scipy.stats as stats

In [55]:
from tqdm import tqdm

In [56]:
from itertools import product

In [57]:
import matplotlib.pyplot as plt

In [58]:
BIG_RESULT_MEMORY_PLACE = []

In [59]:
N_TIMESTEPS = 1800
SIG_KERNEL_SAMPLES_NO = 120
M_RUNS = 100
COVS = [0, 0.05, 0.1, 0.2, 0.3, 0.4]
PROC_DIV = 8
RBF_SIGMA = 2

In [60]:
assert N_TIMESTEPS % SIG_KERNEL_SAMPLES_NO == 0, 'Cannot reshape such array properly'

In [61]:
processes = gen_processes.gen_ar_iid_samples_burn_in(N_TIMESTEPS+1, M_RUNS)

In [62]:
plt.plot(processes[0, :, 0])
plt.plot(processes[0, :, 1]);

In [63]:
def get_bootstrap_samples_lead_lag(X_raw, Y_raw):
    single_sample_length = int(N_TIMESTEPS / SIG_KERNEL_SAMPLES_NO)
        
    x_bootstrap_samples = torch.zeros(SIG_KERNEL_SAMPLES_NO, single_sample_length, 2)
    y_bootstrap_samples = torch.zeros(SIG_KERNEL_SAMPLES_NO, single_sample_length, 2)
    
    rng = np.random.default_rng(1234)
    
    for i in range(SIG_KERNEL_SAMPLES_NO):
        idx_low = rng.integers(0, N_TIMESTEPS - single_sample_length + 1)
        x_bootstrap_samples[i, :, 0] = X_raw[idx_low+1 : idx_low+single_sample_length+1]
        x_bootstrap_samples[i, :, 1] = X_raw[idx_low : idx_low+single_sample_length]
        y_bootstrap_samples[i, :, 0] = Y_raw[idx_low+1 : idx_low+single_sample_length+1]
        y_bootstrap_samples[i, :, 1] = Y_raw[idx_low : idx_low+single_sample_length]

    return x_bootstrap_samples, y_bootstrap_samples

In [64]:
hsic_results = {}
cov_results = {}

In [ ]:
for cov in COVS:#[0, 0.3, 0.8]:    
    processes = gen_processes.gen_ar_iid_samples_burn_in(N_TIMESTEPS+1, M_RUNS, corr=cov)
    hsic_results[cov] = []
    cov_results[cov] = []
    for run in range(M_RUNS):
        X_raw = processes[run, :, 0]
        Y_raw = processes[run, :, 1]
        X_lead_lag_chopped, Y_lead_lag_chopped = get_bootstrap_samples_lead_lag(X_raw, Y_raw)
        
        X = X_raw[:N_TIMESTEPS]
        Y = Y_raw[:N_TIMESTEPS]
        
        hsic_test_result = hsic_independence_test.get_test_result_matrix_shuffle_u_stat(X_lead_lag_chopped/PROC_DIV, Y_lead_lag_chopped/PROC_DIV, static_kernel = sigkernel.RBFKernel(sigma = RBF_SIGMA))
        cov_test_result = cov_independence_test.get_test_result(X, Y)

        print('RUN:', run)
        print('HSIC:', hsic_test_result[1:])
        print('COV:', cov_test_result)

        hsic_results[cov].append(hsic_test_result)
        cov_results[cov].append(cov_test_result)
        
    print('==================== END COV {} ================='.format(cov))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:58<00:00,  4.77s/it]


RUN: 0
HSIC: (-0.0001346742214869412, 0.8)
COV: (tensor(-0.0791), 0.6976861551406668)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:59<00:00,  4.80s/it]


RUN: 1
HSIC: (-4.2255165580762366e-05, 0.42)
COV: (tensor(0.0668), 0.32077175321573415)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:58<00:00,  4.77s/it]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:42<00:00,  4.45s/it]


RUN: 13
HSIC: (0.0024140378081356495, 0.0)
COV: (tensor(0.8312), 1.8892554187743826e-10)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [03:36<00:00,  4.33s/it]


RUN: 14
HSIC: (0.0007229863284510252, 0.0)
COV: (tensor(0.9000), 5.2825965823899423e-11)


 28%|██████████████████████████████▌                                                                              | 14/50 [01:03<02:45,  4.59s/it]

In [ ]:
def get_null_rejection_rate(results):
    a = 0
    for tup in results:
        a += (tup[2] < 0.025) or (tup[2] > 0.975)
    return a / len(results)

In [ ]:
def get_null_sampling_approx_from_all_results(all_results):
    null_approx = None
    for cov in all_results.keys():
        for tup in all_results[cov]:
            if null_approx == None:
                null_approx = tup[0]
            else:
                null_approx = torch.cat((null_approx, tup[0]), dim = 0)
    null =  np.array([item.item() for item in null_approx])
    null.sort()
    return null

In [ ]:
def get_test_statistic_dists_from_all_results(all_results):
    hsic_dist = {}
    a = 0
    for cov in all_results.keys():
        hsic_dist[cov] = []
        for tup in all_results[cov]:
            hsic_dist[cov].append(tup[1])
        hsic_dist[cov] = np.array(hsic_dist[cov])
    return hsic_dist

In [ ]:
def get_test_cutoff_thresholds_for_null(null):
    return (null[int((len(null)-1)*0.025)], null[int((len(null)-1)*0.975)])

In [ ]:
null = get_null_sampling_approx_from_all_results(hsic_results)
test_statistic_dists = get_test_statistic_dists_from_all_results(hsic_results)
low_thresh, high_thresh = get_test_cutoff_thresholds_for_null(null)

In [ ]:
print(get_null_rejection_rate(hsic_results[0.3]))

In [ ]:
plt.hist(null, bins = 100, density = True);
plt.hist(test_statistic_dists[0.3], bins = 20, density = True, alpha = 0.8)
plt.axvline(low_thresh, alpha = 0.3, color = 'red')
plt.axvline(high_thresh, alpha = 0.3, color = 'red')

In [ ]:
def push_results():
    BIG_RESULT_MEMORY_PLACE.append({
        'N_TIMESTEPS' : N_TIMESTEPS,
        'SIG_KERNEL_SAMPLES_NO' : SIG_KERNEL_SAMPLES_NO,
        'M_RUNS' : M_RUNS,
        'COVS' : COVS,
        'PROC_DIV' : PROC_DIV,
        'RBF_SIGMA' : RBF_SIGMA,
        'NULL' : null,
        'TEST DISTS' : test_statistic_dists,
        'TEST HIGH LOW THRES': (high_thresh, low_thresh)
    })
    pp.pprint(BIG_RESULT_MEMORY_PLACE[-1])

In [ ]:
push_results()

In [ ]:
import datetime

In [ ]:
import pickle
with open('./corr_vs_hsic_comparison_BIG_RESULTS_{}.pickle'.format(datetime.datetime.today()).replace(' ', '_').replace(':', '.'), 'wb') as f:
    pickle.dump(BIG_RESULT_MEMORY_PLACE, f)